In [1]:
# 分析时间：2013年1月1日-2022年12月31日，总共10年时间
# 但是在做特征分析的时候，可以把数据拉长。多搞一年：即2012-01-01至2022-12-31
feature_start_date_str = "2011-01-01"
start_date_str = "2012-01-01"
end_date_str = "2022-12-31"

In [2]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import QuantLib as ql
%matplotlib inline

plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

In [3]:
def clean_data(raw_df, date_benchmark, feature_start_date, end_date):
    clean_df = raw_df[(raw_df['Date']>=feature_start_date) & (raw_df['Date']<=end_date)].reset_index(drop=True)
    investment_fac_df = pd.merge(date_benchmark, clean_df, on="Date", how="left")
    # 采用向下填充
    clean_df = clean_df.fillna(method='ffill')
    return clean_df

def plot_features(clean_df, col_name_list, title="Investment in fixed assets completed"):
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for temp_col in col_name_list:
        ax.plot(clean_df['Date'], clean_df[temp_col], label=temp_col)
        

    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel("Value")
    fig.autofmt_xdate()
    plt.tight_layout()
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [4]:
feature_start_date = datetime.datetime.strptime(feature_start_date_str, "%Y-%m-%d")
start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")
print(feature_start_date, start_date, end_date)

2011-01-01 00:00:00 2012-01-01 00:00:00 2022-12-31 00:00:00


In [5]:
# 取出目标数据
# 到期收益率
raw_yield_to_maturity_df = pd.read_excel('./data/国债到期收益率.xlsx')
raw_yield_to_maturity_df = raw_yield_to_maturity_df.rename(columns={'指标名称': 'Date',
       '国债到期收益率:6个月': "6M", '国债到期收益率:1年': "1Y", '国债到期收益率:2年': "2Y", '国债到期收益率:3年': "3Y",
       '国债到期收益率:4年': "4Y", '国债到期收益率:5年': "5Y", '国债到期收益率:6年': "6Y", '国债到期收益率:7年': "7Y", 
        '国债到期收益率:8年': "8Y",'国债到期收益率:9年': "9Y", '国债到期收益率:10年': "10Y", '国债到期收益率:15年': "15Y", 
        '国债到期收益率:20年': "20Y", '国债到期收益率:30年': "30Y"})
raw_yield_to_maturity_df

,Date,6M,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,15Y,20Y,30Y
0,2008-07-01,3.2769,3.4474,3.8065,4.0301,4.1697,4.3093,4.3498,4.3903,4.4292,4.4682,4.5072,4.9618,4.9639,4.9498
1,2008-07-02,3.4055,3.5461,3.8015,3.9506,4.0883,4.2260,4.3019,4.3778,4.4282,4.4787,4.5291,4.9442,4.9462,4.9501
2,2008-07-03,3.3343,3.4972,3.8687,3.9820,4.1085,4.2350,4.3163,4.3976,4.4436,4.4896,4.5356,4.9453,4.9469,4.9501
3,2008-07-04,3.3397,3.4763,3.8179,3.9854,4.1031,4.2209,4.3183,4.4157,4.4618,4.5080,4.5542,4.9469,4.9479,4.9501
4,2008-07-07,3.3679,3.4796,3.8138,3.9561,4.0850,4.2140,4.2849,4.3559,4.4215,4.4871,4.5527,4.9226,4.9285,4.9404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,2023-03-13,2.1958,2.2696,2.4373,2.5289,2.6127,2.6965,2.7655,2.8345,2.8473,2.8600,2.8728,3.0310,3.1147,3.2772
3668,2023-03-14,2.1950,2.2655,2.4475,2.5215,2.6046,2.6877,2.7591,2.8305,2.8417,2.8528,2.8640,3.0196,3.1047,3.2692
3669,2023-03-15,2.2050,2.2638,2.4432,2.5278,2.6121,2.6963,2.7663,2.8363,2.8474,2.8584,2.8695,3.0152,3.1033,3.2700
3670,2023-03-16,2.1799,2.2598,2.4193,2.5182,2.5953,2.6723,2.7483,2.8242,2.8342,2.8441,2.8541,2.9998,3.0857,3.2503


In [6]:
# 按特征长度选择数据
yield_to_maturity_df = raw_yield_to_maturity_df[(raw_yield_to_maturity_df['Date']>=feature_start_date)
                                                & (raw_yield_to_maturity_df['Date']<=end_date)].reset_index(drop=True)
yield_to_maturity_df

,Date,6M,1Y,2Y,3Y,4Y,5Y,6Y,7Y,8Y,9Y,10Y,15Y,20Y,30Y
0,2011-01-04,2.9635,3.0310,3.3571,3.3839,3.4591,3.5343,3.6066,3.6790,3.7370,3.7951,3.8531,4.0629,4.1640,4.2839
1,2011-01-05,2.9262,3.0047,3.1834,3.2891,3.3696,3.4500,3.5466,3.6432,3.6993,3.7553,3.8113,4.0108,4.1683,4.2554
2,2011-01-06,2.6401,2.8361,3.1205,3.2438,3.3567,3.4696,3.5691,3.6686,3.7146,3.7607,3.8067,4.0104,4.1645,4.2528
3,2011-01-07,2.6055,2.7599,3.1324,3.3165,3.3916,3.4667,3.5586,3.6505,3.7163,3.7820,3.8478,4.0208,4.1638,4.2649
4,2011-01-10,2.5607,2.7767,3.0750,3.2557,3.3469,3.4381,3.5293,3.6205,3.6891,3.7578,3.8264,4.0104,4.1569,4.2575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2988,2022-12-26,2.1778,2.2398,2.3669,2.4616,2.5396,2.6175,2.7175,2.8175,2.8249,2.8324,2.8398,2.9777,3.0579,3.2225
2989,2022-12-27,2.2125,2.2615,2.3889,2.4898,2.5777,2.6656,2.7526,2.8395,2.8475,2.8556,2.8636,3.0014,3.0676,3.2182
2990,2022-12-28,2.2398,2.2732,2.4056,2.4746,2.5771,2.6795,2.7595,2.8395,2.8496,2.8598,2.8699,3.0078,3.0679,3.2125
2991,2022-12-29,2.1737,2.1952,2.3605,2.4405,2.5415,2.6425,2.7314,2.8202,2.8255,2.8309,2.8362,2.9741,3.0459,3.2021


In [7]:
date_benchmark = yield_to_maturity_df[["Date"]]
date_benchmark

,Date
0,2011-01-04
1,2011-01-05
2,2011-01-06
3,2011-01-07
4,2011-01-10
...,...
2988,2022-12-26
2989,2022-12-27
2990,2022-12-28
2991,2022-12-29


In [8]:
### 上证指数
raw_sh_df = pd.read_excel("./data/features/000001.SH-行情统计-20230318.xlsx")
raw_sh_df = raw_sh_df.rename(columns={
    '交易日期': "Date", '开盘点位': "SH_Open", '最高点位': "SH_High", '最低点位': "SH_Low", 
    '收盘价': "SH_Close", '涨跌': "SH_Change", '涨跌幅(%)': "Change_Rate", '开始日累计涨跌': "SH_Cum_Change",
       '开始日累计涨跌幅': "SH_Cum_Change_Rate", '成交量(万股)': "SH_Volume", '成交额(万)': "SH_Amount"
})
raw_sh_df['Date'] = pd.to_datetime(raw_sh_df['Date'])
raw_sh_df

,Date,SH_Open,SH_High,SH_Low,SH_Close,SH_Change,Change_Rate,SH_Cum_Change,SH_Cum_Change_Rate,SH_Volume,SH_Amount
0,2023-03-17,"3,245.26","3,279.92","3,240.16","3,250.55",23.65,0.73,"3,150.55","3,150.55","3,430,010.22","43,198,305.77"
1,2023-03-16,"3,244.47","3,257.43","3,224.72","3,226.89",-36.42,-1.12,"3,126.89","3,126.89","3,273,896.32","39,291,020.84"
2,2023-03-15,"3,261.03","3,273.12","3,256.86","3,263.31",18.00,0.55,"3,163.31","3,163.31","3,051,687.98","37,327,638.44"
3,2023-03-14,"3,261.92","3,262.00","3,216.99","3,245.31",-23.38,-0.72,"3,145.31","3,145.31","3,199,060.33","40,997,627.61"
4,2023-03-13,"3,228.12","3,268.70","3,228.12","3,268.70",38.62,1.20,"3,168.70","3,168.70","2,949,280.87","36,441,420.65"
...,...,...,...,...,...,...,...,...,...,...,...
7872,1990-12-25,120.09,120.25,114.55,120.25,5.70,4.98,20.25,20.25,0.15,0.65
7873,1990-12-24,113.57,114.55,109.13,114.55,5.42,4.97,14.55,14.55,0.32,3.11
7874,1990-12-21,109.07,109.13,103.73,109.13,4.74,4.54,9.13,9.13,0.28,1.61
7875,1990-12-20,104.30,104.39,99.98,104.39,4.39,4.39,4.39,4.39,1.97,8.50


In [9]:
sh_df = clean_data(raw_sh_df, date_benchmark, feature_start_date, end_date)
sh_df

,Date,SH_Open,SH_High,SH_Low,SH_Close,SH_Change,Change_Rate,SH_Cum_Change,SH_Cum_Change_Rate,SH_Volume,SH_Amount
0,2022-12-30,"3,084.52","3,096.31","3,082.20","3,089.26",15.56,0.51,"2,989.26","2,989.26","2,175,453.44","25,035,595.09"
1,2022-12-29,"3,076.73","3,086.00","3,064.46","3,073.70",-13.70,-0.44,"2,973.70","2,973.70","2,155,706.76","25,391,698.12"
2,2022-12-28,"3,088.62","3,098.65","3,079.43","3,087.40",-8.17,-0.26,"2,987.40","2,987.40","2,245,541.51","26,050,852.57"
3,2022-12-27,"3,077.75","3,098.08","3,074.31","3,095.57",30.01,0.98,"2,995.57","2,995.57","2,222,183.22","26,794,678.54"
4,2022-12-26,"3,048.20","3,071.84","3,047.35","3,065.56",19.70,0.65,"2,965.56","2,965.56","2,065,038.93","25,547,026.53"
...,...,...,...,...,...,...,...,...,...,...,...
2912,2011-01-10,"2,832.62","2,847.36","2,786.86","2,791.81",-46.99,-1.66,"2,691.81","2,691.81","907,808.72","11,079,677.14"
2913,2011-01-07,"2,820.67","2,868.01","2,807.87","2,838.80",14.60,0.52,"2,738.80","2,738.80","1,131,295.68","13,716,350.06"
2914,2011-01-06,"2,838.44","2,853.88","2,816.95","2,824.20",-14.40,-0.51,"2,724.20","2,724.20","847,056.16","11,325,158.99"
2915,2011-01-05,"2,833.12","2,850.49","2,824.30","2,838.59",-14.06,-0.49,"2,738.59","2,738.59","1,044,152.82","13,526,479.99"


In [10]:
for temp in sh_df.columns.tolist()[1:]:
    sh_df[temp] = sh_df[temp].astype(str).str.replace(",", '')
    sh_df[temp]  = sh_df[temp].astype(float)

In [11]:
sh_df.to_csv("./data/clean_features/features_stock_market.csv", index=False, encoding="utf-8-sig")

In [12]:
sh_df.describe()

,SH_Open,SH_High,SH_Low,SH_Close,SH_Change,Change_Rate,SH_Cum_Change,SH_Cum_Change_Rate,SH_Volume,SH_Amount
count,2917.000000,2917.000000,2917.000000,2917.000000,2917.000000,2917.000000,2917.000000,2917.000000,2.917000e+03,2.917000e+03
mean,2918.569383,2940.790185,2896.933178,2921.604553,0.096373,0.011724,2821.604542,2821.604529,2.138195e+06,2.469269e+07
std,539.915508,546.069766,531.864963,540.387415,41.249634,1.293176,540.387418,540.387440,1.323897e+06,1.807383e+07
min,1935.520000,1959.160000,1849.650000,1950.010000,-345.350000,-8.490000,1850.010000,1850.010000,3.963258e+05,3.311072e+06
25%,2437.470000,2452.990000,2426.700000,2439.630000,-15.920000,-0.560000,2339.630000,2339.630000,1.151197e+06,1.094381e+07
50%,2987.180000,3009.200000,2968.180000,2988.090000,1.290000,0.050000,2888.090000,2888.090000,1.768521e+06,1.960361e+07
75%,3277.810000,3297.690000,3261.610000,3281.740000,17.890000,0.620000,3181.740000,3181.740000,2.911722e+06,3.456834e+07
max,5174.420000,5178.190000,5103.400000,5166.350000,224.190000,5.760000,5066.350000,5066.350000,8.571328e+06,1.309925e+08
